In [ ]:
%pip install polars pyarrow

In [64]:
import asyncio
import os
import json

import pandas as pd
import httpx
import polars as pl
from web3 import Web3
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
df = pl.read_csv("./data/whales.csv")
whales = df.select(["ownerAddress"])
whales

In [ ]:
INFURA_ETH_MAINNET_PROVIDER = os.getenv("INFURA_ETH_MAINNET_PROVIDER")
w3 = Web3(Web3.HTTPProvider(INFURA_ETH_MAINNET_PROVIDER))
w3.is_connected()

In [ ]:
headers = {"Content-Type": "application/json"}
rpc_json = lambda address: {
    "jsonrpc": "2.0",
    "method": "eth_getCode",
    "params": [address, "latest"],
    "id": 1
}

def get_code(address):
    try:
        rpc = rpc_json(address)
        response = httpx.post(INFURA_ETH_MAINNET_PROVIDER, headers=headers, json=rpc)
    except:
        return None
    
    return response.json()["result"]

In [ ]:
df.with_columns(
    [
        pl.col("ownerAddress").map_elements(get_code).alias("code")
    ]
)
df

In [ ]:
df

In [ ]:
async def get_code(address, session):
    try:
        rpc = rpc_json(address)
        response = await session.post(INFURA_ETH_MAINNET_PROVIDER, headers=headers, json=rpc)
    except Exception as e:
        print(e)
        return None
    
    return response.json()["result"]

In [73]:
df

ownerAddress,tokenCount,estimated_price
str,i64,f64
"""0x29469395eAf6…",4195,18861.488602
"""0x3eb879cc9a0E…",5612,13353.793545
"""0xaAa2DA255DF9…",1077,12764.981954
"""0x7285e8F0186a…",413,11569.937303
"""0x44d97D22B3d3…",266,9837.411983
"""0x1c593c369136…",562,8225.904148
"""0xe2A83b15FC30…",400,6051.218799
"""0xFA3CE71036dd…",1818,6047.266793
"""0x24e267D13314…",104,5535.894198


In [ ]:
whales = df["ownerAddress"].to_list()

async with httpx.AsyncClient() as session:
    whales = await asyncio.gather(*[get_code(address, session) for address in whales])

In [74]:
new_df = pl.DataFrame({
    "owner_address": df["ownerAddress"].to_list(),
    "code": whales,
    "estimated_price": df["estimated_price"].to_list(),
})

new_df

owner_address,code,estimated_price
str,str,f64
"""0x29469395eAf6…","""0x608060405236…",18861.488602
"""0x3eb879cc9a0E…","""0x608060405236…",13353.793545
"""0xaAa2DA255DF9…","""0x608060405260…",12764.981954
"""0x7285e8F0186a…","""0x608060405260…",11569.937303
"""0x44d97D22B3d3…","""0x608060405234…",9837.411983
"""0x1c593c369136…","""0x""",8225.904148
"""0xe2A83b15FC30…","""0x""",6051.218799
"""0xFA3CE71036dd…","""0x608060405236…",6047.266793
"""0x24e267D13314…","""0x608060405273…",5535.894198


In [75]:
%pip install openpyxl
df_whales = pd.DataFrame(data=new_df.to_dicts())
df_whales.to_excel("./data/whales_with_code.xlsx", index=False)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
